# PACKAGES

In [ ]:
import get_data as gd
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import Asset as a
import numpy as np
import random
from ta.momentum import RSIIndicator
import matplotlib.pyplot as plt

In [ ]:
# Grab bitcoin data
df_data = gd.grab_data()

# Hyperparameter

In [ ]:
NUMBER_OF_GENERATION = 3
POPULATION_SIZE = 2000
CROSSOVER_PROBABILITY = 0.5
MUTATION_PROBABILITY = 0.3
HIGHEST_WINDOWSIZE = 30
LOWEST_WINDOWSIZE = 4
HIGHEST_HIGHTHRESHOLD = 90
LOWEST_HIGHRESHOLD = 70
HIGHEST_LOWTHRESHOLD = 30
LOWEST_LOWTHRESHOLD = 10
MUTATION_HIGH = 1.3
MUTATION_LOW = 0.7

#plotting hyperparams
N_COLS = 2
N_PLOTS = NUMBER_OF_GENERATION + 1
N_ROWS = (N_PLOTS + N_COLS - 1) // N_COLS

# Trade Strategy

In [ ]:
def apply_strategy(data, individual, start_time, end_time):
    # start_time is not allowed to be 0 since the close price yesterday is needed
    if start_time <= 0:
        raise ValueError("start time is not allowed to be 0")
    # Initialize some variables
    windowsize, highthreshold, lowthreshold = individual
    rsi =RSIIndicator(data["close"], window=windowsize)
    data["rsi"] = rsi.rsi()
    btc = np.asarray(data["close"])
    asset = a.Asset()
    in_position = False
    money_progress = []
    buy_days = []
    sell_days = []

    # Apply buy and sell strategy 
    for i in range(start_time - 1, end_time -1):
        #buy
        if not in_position and data["rsi"].iloc[i] < lowthreshold:
            asset.buy(btc[i+1])
            in_position =True
            buy_days.append(i)
        #sell
        elif in_position and data["rsi"].iloc[i] > highthreshold:
            asset.sell(btc[i+1])
            in_position =False
            sell_days.append(i)
        #store the value of money in each iteration
        money_progress.append(asset.money + asset.coin * btc[i+1])

    if in_position:
        asset.sell(btc[end_time])
        sell_days.append(end_time)

    money_progress.append(asset.money)

    btc_prices = btc[start_time:end_time +1]
    return money_progress, btc_prices, buy_days, sell_days

#calculate fitness of each individual 
def calculate_fitness(data. individual, start_time, end_time):
    money_progress, btc_prices, buy_days, sell_days = apply_strategy(data, individual, start_time, end_time)
    score = money_progress[-1]
    return score

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

# calculate the probability of each individual and select them as the parents of next generation
def roulette_wheel_selection(population, fitness):
    softmax_fitness = softmax(fitness)
    selected_indices = np.random.choice(range(len(population)), size=2, replace=False, p=softmax_fitness)
    return [population[i] for i in selected_indices]

# PLOT

In [ ]:
def draw_top_individuals(top_individuals, start_time, end_time, generation, is_test):
    #plot the money progress of the top 4 individuals in last generation
    color_list = ['r', 'b', 'g', 'k']
    subplot_location = generation // N_COLS, generation % N_COLS

    #plot.figure(figsize=(8,4))
    for idx, (individual, fit) in enumerate(top_individuals[:4]):
        window, hight, lowt = individual
        days =range(start_time, end_time+1)
        money_progress, btc_prices, buy_days, sell_days = apply_strategy(df_data, individual, start_time, end_time)

        # when it is test dataset
        if is_test:
            # plot asset line
            axes[subplot_location].plot(days, money_progress, color=color_list[idx], label=f"Individual {idx + 1}")

            # Plot buy and sell points
            for buy_day in buy_days:
                axes[subplot_location].plot(buy_day, money_progress[buy_day - start_time], 'go', markersize=5)
            for sell_day in sell_days:
                axes[subplot_location].plot(sell_day, money_progress[sell_day - start_time],'ro', markersize=5)
            
            #plot the btc_prices
            ax2 = axes[subplot_location].twinx()
            ax2.plot(days, btc_prices, linestyle='--', color='orange', alpha=0.5, label='BTC Price (AUD)')
            ax2.set_ylabel("BTC Price (AUD")

            # Add legend
            buy_marker = plt.Line2D([], [], color='g', marker='o', linestyle='None', markersize=5, label='Buy')
            sell_marker = plt.Line2D([], [], color='r', marker='o', linestyle='None', markersize=5, label='Sell')
            btc_price_line = plt.Line2D([], [], color='orange', linestyle='--', alpha=0.5, label='BTC Price (AUD)')
            axes[subplot_location].legend(handles=[buy_marker, sell_marker, btc_price_line], loc='best')

        else:
            axes[subplot_location].plot(days, money_progress, color=color_list[idx], 
                                    label=f"Individual {idx + 1}: | {window:02d} | {hight:03.2f} | {lowt:03.2f} |")
            axes[subplot_location].legend(loc='upper left')
            
    axes[subplot_location].set_title(f"Generation {generation + 1}" if generation != NUMBER_OF_GENERATION else 'Test Set')
    axes[subplot_location].set_xlabel("Day")
    axes[subplot_location].set_ylabel("Money(AUD)")

    # plt.savefig("figure/Top4IndividualsMoneyProgress.png")
    # plt.show()


# GENETIC ALGORITHM

In [ ]:
def create_population(size):
    population = []
    # Generate population by random parameters(genes)
    for _ in range(size):
        windosize = random.randint(LOWEST_WINDOWSIZE, HIGHEST_WINDOWSIZE)
        highthreshold = random.uniform(LOWEST_HIGHRESHOLD, HIGHEST_HIGHTHRESHOLD)
        lowthreshold = random.uniform(LOWEST_LOWTHRESHOLD, HIGHEST_LOWTHRESHOLD)
        individual = (windosize, highthreshold, lowthreshold)
        population.append(individual)
    return population